# Mapping Participation

This project is based on actual data analysis that I regularly perform as part of my Head of IT role at BALTIC Centre for Contemporary Art. 

![Map pin](Images/mappin.png?raw=true)

Baltic is a free to enter Contemporary Art Gallery on the banks of the River Tyne in Gateshead with charity status. The Gallery has an educational programme to accompany its Arts programme providing a variety of artist lead workshops both within the gallery and within the broader community. Off-site workshops includes pop-up acivities at regional festivals and events as well as activities in local libraries and children's centres.

The residents of the Gateshead area are one of BALTIC key audience and as part of BALTIC's partnership with Gateshead Council, ensuring that Gateshead residence engage with BALTIC's artistic programme is therefore a key priority. In order to ensure that targets are met participants of learning activities either on site and off site are asked to provide postcode details when they attend an event. This Data is then analysed.

The Data in this project is sample data and does not relate to actual participants or real activities (I used [doogal](https://www.doogal.co.uk/BatchReverseGeocoding.php) to get postcodes from random logitude and latitude co-ordinates. 

From landmark locations to participant postcodes, many dataset contain some form of geographical data. Being able to visualise this data is often key to understanding the relationship between the datapoints and mapping provides a great way to do this. 

When it comes to maps there are an increasing number of options, from [googlemaps](https://www.google.co.uk/maps) and [openstreet maps](https://www.openstreetmap.org/#map=5/54.910/-3.432) to figurative maps and heatmaps. 


## Working with Postcodes

Most of us will will not know the longatude and latitudes of a given location, but we may well know the postcode, for this reason its more common to capture postcodes than any other form of location data.  

Postcode data however is not terribly useful in its own right, postcode areas highly irregular, different postcodes are different sizes and postcode sysyem is not layed out in a any form of regular pattern. It is therefore very difficult to get insights into geographical groupings from post code alone.

## Getting Co-ordinates

Before we can start plotting locations it is therefore necessary to be able to convert postcode into some form of co-ordinate system. There are several co-ordinate systems to choose. For example longatude and latitudes or grid reference.

There are many ways that you can get x and y co-ordinates from postcodes. I used a [freemaptools](https://www.freemaptools.com/convert-uk-postcode-to-lat-lng.htm)

If you wanted you could automate this process by using google's [geocoding api](https://developers.google.com/maps/documentation/geocoding/intro#geocoding) for developers.

# Visualizing Learning participation on Google Maps

The first stage in analysing learning participation was to visualise where participants live by plotting their geographical data on google maps. In order to plot locations on google maps, postcode data must first the converted to latitude and longitude. In the first instance this was performed as an excel vlookup based on data taken from Data.gov.uk relating to regional parlimentary seats. https://www.freemaptools.com/convert-uk-postcode-to-lat-lng.htm was used to get latitude and longitude details for post codes outside of the region.

The data was loaded to Google maps as 3 separate csv files, each .csv file being imported into its own layer within which can be individually styled and labelled.

![Google Maps](Images/googlemaps.jpeg?raw=true)

The first of the .csv files contained the location data for the venues at which each of the learning activities took place. The second .csv contained information about participants and activities that took place with the Gallery itself and the final .csv caontained the same information for events that took place at off-site locations. The participant information included the dates times and seasons of the different activities, all of which can be used define the colours, styles and visibility of the markers on the map in order to visualize participation patterns. 

The boundaries of the Gatehead Local Authority area were downloaded as a .kml file from https://mapit.mysociety.org/area/2523.html. This .kml was then imported into a layer within google maps which clearly identify participation by residents of the metropolitan borough area on the map.


![GoogleMap 2](Maps/learningparticipation20182.png?raw=true)


In addition to providing useful monitoring information about the participation in arts activities within the Map data . The participation map can be used as a planning tool to help develope future offsite learning activities aimed at targeting areas not currently being reach by activities undertaken within the gallery. 

# Plotting Offsite Activity with Matplotlib

Longitude and Latitude basically just x and Y values. If your datapoints are reasonably close together you can plotted them on a very basic level as a scatter plot with matplotlib, seaborn , Bokeh or any other drawing/plotting library.

The Earth is of course a sphere, and if your datapaoints are further apart or you want to plot them in more accuracy you would need to adjust for this. In terms of initial exploratory data analysis however a scatter plot is a very quick way to get an idea of the geographical distribution of your data points.

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab

# import the data into a dataframe
df=pd.read_csv('Data/Venues.csv', sep=',')
print(df.head())

# create a plot

plt.scatter(df['Longitude'],df['Latitude'], s=50, c='red')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

pylab.savefig('Maps/basicplot.png')
plt.show()

# Creating a  Venue Map with Folium

[Folium](https://github.com/python-visualization/folium) is a library Module that allows you to plot maps in python. In this example we've created a basic map with Folium, set the default view and passed the longitude , latitude and venue names as markers. We'll then go on to produce cluster maps and a headmap.

While a scatter plot gives you the skeleton of an idea of the Venues locations relative to each other. Plotting this on a map allows you to see how that relates to other geographic features like schools, public public ammenities and transport networks.

![Folium Map](Images/foliummap.png?raw=true)

In [ ]:
# Create a map using the folium module

# import libraries
import folium
import pandas as pd
import pylab

# import the data into a dataframe
df=pd.read_csv('Data/Venues.csv', sep=',')

# set default position and scale
lon, lat = -1.49828,   54.79921
zoom_start = 10

# initiate map
venue_map = folium.Map(tiles='OpenStreetMap',zoom_start=zoom_start,control_scale=True)

# create datapoints with loop
allvenues=[]
for i, row in df.iterrows():
    venue={'loc': [row['Latitude'], row['Longitude']], 'label':row['Venue']}
    allvenues.append(venue)

# Add markers to the map
for venue in allvenues:
    marker = folium.Marker(location=venue['loc'], popup=venue['label'])
    marker.add_to(venue_map)

# The last object in the cell always gets shown in the notebook
pylab.savefig('Maps/folium.png')
venue_map.save('Maps/foliumscale.html')
venue_map

# Mapping Participation

Now that we know where the activities are taking place, we can move on to take a look at where the participants are coming from. We'll use the a slight variation of the code to plot small map pins for the venues and circles for the participant locations. So that we understand which pins represent participant of which activities, we'll use 'Activity' as the label and we'll colour Gallery participant red and offsite participants blue.

![ParticipantMap](Maps/participants.png?raw=true)


In [ ]:
# Create a participant map using the folium module

# import libraries
import folium
import pandas as pd

# import the data into a dataframe
Venuesdf=pd.read_csv('Data/Venues.csv', sep=',')
Participantsdf=pd.read_csv("Data/All_Activities.csv", encoding = "ISO-8859-1", sep=',')

# seperate Gallery and offsite dataframes
Gallerydf=Participantsdf[Participantsdf['Venue']=="BALTIC"]
Offsitedf=Participantsdf[Participantsdf['Venue']!="BALTIC"]

# initiate map
participant_map = folium.Map(location=[54.79921, -1.49828],tiles='OpenStreetMap',zoom_start=10,control_scale=True)

# create venue datapoints with loop
allvenues=[]
for i, row in Venuesdf.iterrows():
    venue={'loc': [row['Latitude'], row['Longitude']], 'label':row['Venue']}
    allvenues.append(venue)

# Add venue markers to the map
for venue in allvenues:
    marker = folium.Marker(location=venue['loc'], popup=venue['label'])
    marker.add_to(participant_map)
    
# create gallery participant and offsite participant datapoints with loops
offsiteparticipants=[]
for i, row in Offsitedf.iterrows():
    offsiteparticipant={'loc': [row['attend Latitude'], row['attend Longitude']], 'label':row['Session']}
    offsiteparticipants.append(offsiteparticipant)

galleryparticipants=[]
for i, row in Gallerydf.iterrows():
    galleryparticipant={'loc': [row['attend Latitude'], row['attend Longitude']], 'label':row['Session']}
    galleryparticipants.append(galleryparticipant)
 
# colours red: #CC0033 blue: #43d9de
# Add offsite participant markers to the map
for offsiteparticipant in offsiteparticipants:
    omarkers = folium.CircleMarker(location=offsiteparticipant['loc'], popup=offsiteparticipant['label'],fill=True,fill_color='#CC0033',color='#CC0033',radius=3)
    omarkers.add_to(participant_map)  
    
# Add gallery participant markers to the map
for galleryparticipant in galleryparticipants:
    gmarkers = folium.CircleMarker(location=galleryparticipant['loc'], popup=galleryparticipant['label'],fill=True,fill_color='#43d9de',color='#43d9de',radius=3)
    gmarkers.add_to(participant_map) 

# The last object in the cell always gets shown in the notebook
participant_map.save('Maps/foliumparticipants.html')
participant_map

# Identifying Communities with Cluster Maps

Interactively mapping participant locations with pop up labels and colour coded markers to show Venue or Activity, is a really great way to get a very detailed insight into individual participants. The more datapoints you add though the more cluttered the map becomes, until you reach the point where it becomes simply impractical to view participant data in this way.

In many ways knowing the exact locations of individual participants while being interesting is not entirely useful. What would be more help would be knowing where groups of participants or communities are located. Once you can locate communities you can start to target those groups either with messages like local bus stop Ads or by engaging with  the communities through offsite activities. You can also use this information to identify similar communities that may have the same interests

![Cluster Map](Maps/clusters.png?raw=true)

In [ ]:
# Participant Cluster Map

from folium import plugins
from folium.plugins import MarkerCluster

# import data
Participantdf=pd.read_csv('Data/All_Activities.csv', sep=',')
 
cluster_map = folium.Map(location=[54.79921, -1.49828],zoom_start=10)
marker_cluster = folium.plugins.MarkerCluster().add_to(cluster_map)

for name, row in Participantdf.iterrows():
    folium.Marker([row['attend Latitude'], row['attend Longitude']], popup="{0}: {1}".format(row["Venue"], row["Session"])).add_to(marker_cluster)
cluster_map.save('Maps/Cluster.html')
cluster_map

# Visualising Participant Concentration with a heatmap

We previously used a Cluster Map to declutter the map and help us identify areas with clusters of participants, this is very helpful us identify communities of participants. Next we'll use a heat map.

Heatmaps allow us to visualize communities in a slightly different way. Instead of arranging participants into  clusters and representing each cluster as a unique point on a map, often colour coding this to symbolise number of participants, heatmaps use a more generic colour overlay, where areas with the same density of datapoints (participants) are represented by the same colour. The colours are typically continuous colour ranges. The most common  range goes from hot colours like red, through warm colours like orange, to neutral colours like yellows and greens to cold colours like blue and purple.

Because Heatmaps show more general concentrations of participants rather than individual data points, its harder to use labels or to see secondary detils, heatmaps also have a tendancy to obscure the underlying map data. On the other hand most people instintively know how to read Heatmaps, and you can not only areas of high participation levels, but also areas with very low participation levels

![heatmap](Maps/Heatmap.png?raw=true)


In [ ]:
#HeatMaps

import folium
from folium import plugins

# import the data into a dataframe
Participantsdf=pd.read_csv("Data/All_Activities.csv", encoding = "ISO-8859-1", sep=',')

# initiate map
participation_heatmap = folium.Map(location=[52.00000,-1.59828],tiles='OpenStreetMap',zoom_start=6,control_scale=True)

participation_heatmap.add_child(plugins.HeatMap([[row['attend Latitude'], row['attend Longitude']] for name, row in Participantsdf.iterrows()]))
participation_heatmap.save("Maps/heatmap.html")
participation_heatmap


Looking at the heatmap that we've just produced, you can see unsurprisingly that the strongest concentration of participants in from the North East region (this is where the Gallery is based) and London (which has the highest population generally). There are also hotspots in places like, Wakefield, Nottingham, Manchester, Swansea, Exeter, Peterborogh. At first sight these seem random, but they are all cities with with Strong links to Art, their  Universities have a reputation for Art, and manny have a famous Contemporary Art Gallery of their own.

# Quantifying Participant Numbers

Where we used the .kml file within Google Maps to see the boundary for the Gateshead Local Authority area. We will need a list or dictionary of all of the postcodes considered to be within the Local Authority area in order to establish what percentages of participant are residents of Gateshead statistically or programmatically.

Although there are a large number of online resources for local authority data, we were unable to find a full list of postcodes by Local Authority in a format that could easily be downloaded and imported to python, the majority of data using alternate geographical units. 

Some generic data was available with partial postcode information within each Local authority (eg NE40, NE36 etc). It was possible to match the partial references with full postcodes from the parlimentary seat dataset and then further refinw this list by plotting each of the datapoints on google maps and comparing it with the county boundaries removing any unwanted datapoints manually.

In [ ]:
#import modules
import pandas as pd
import numpy as np

# import all postcodes within GLA Area 
postcodes=pd.read_csv('Data/Gatesheadpostcodes.csv')

# Transfer GLA postcodes into a dictionary
postcodedict = {}
codes = postcodes["postcode"]
for c in codes:
    if c in postcodedict.keys():
        postcodedict[c]=((postcodedict)[c])+1
    else:
        postcodedict[c]=1    

# import onsite participant postcodes
Participantsdf=pd.read_csv("Data/All_Activities.csv", encoding = "ISO-8859-1", sep=',')

# Analyse participant postcodes
# initiate counts
bgateshead=0
bcount=0
bcodes = Participantsdf["Postcode"]

# Iterate over codes in postcodes column
for c in bcodes:
    if c in postcodedict.keys():
        bgateshead=bgateshead+1
    else:
        bcount=bcount+1    

# print results
print("-------------------------------------")
print(str(bgateshead) + " participants of BALTIC onsite learning events came from Gateshead")
print(str(bcount) + " participants of BALTIC onsite learning events came from outside of Gateshead")
print("-------------------------------------")

# import offsite participant postcodes
off_participants=pd.read_csv("Data/Offsite_Activities.csv")

# initiate counts
offgateshead=0
offcount=0
offcodes = off_participants["Postcode"]

# Iterate over codes in postcodes column
for c in offcodes:
    if c in postcodedict.keys():
        offgateshead=offgateshead+1
    else:
        offcount=offcount+1         
        
print(str(offgateshead) + " participants of BALTIC offsite learning events came from Gateshead")
print(str(offcount) + " participants of BALTIC offsite learning events came from outside of Gateshead")
print("-------------------------------------")

# Calculating Average Travel Distances

Because we have the location of the participants and the venues we can very crudely calculate travel distances. You have to do a little maths to compensate for the curvature of the earth and the distance are the direct distance between the 2 points not taking into account the means of travel, obstructions or route choices.

In [ ]:
# Calculate Distances
# import modules
import pandas as pd
from math import sin, cos, sqrt, atan2, radians

# import participant data
Activitiesfile="Data/All_Activities.csv"
Participantsdf=pd.read_csv(Activitiesfile, encoding = "ISO-8859-1", sep=',')
Venuesdf=pd.read_csv('Data/Venues.csv', sep=',')

# Calculate Individula Travel distance and add to distance column
R = 3956 # Radius of earth use 6373.0 for kilometers or 3956 for miles

for index, row in Participantsdf.iterrows():
    dlon =(radians(float(row["venue long"]))-(radians(float(row["attend Longitude"]))))
    dlat =((radians(float(row["venue lat"])))-(radians(float(row["attend Latitude"]))))
    a = sin(dlat / 2)**2 + cos(row["venue lat"]) * cos(row["attend Latitude"]) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    Participantsdf.set_value(index, "Distance", distance, takeable=False)   

# Calculate Average Travel distance for each Venue
print("-------------------------------------")
print("Distance Travelled")
print("-------------------------------------")
print("The shortest distance travelled to attend an activity was " + (str(min((Participantsdf)["Distance"]))) + " miles")
print("The largest distance travelled to attend an activity was " + (str(max((Participantsdf)["Distance"]))) + " miles")
print("The average (mean) distance travelled to attend an activity was " + (str(float((Participantsdf)["Distance"].mean()))) + " miles")
print("The average (median) distance travelled to attend an activity was " + (str(float((Participantsdf)["Distance"].median()))) + " miles")
# print("The average (mode) distance travelled to attend an activity was " + (str(float((Participantsdf)["Distance"].mode()))) + " miles")
print("-------------------------------------")  

# Calculate Average Distance
AllAvgDists=[]
for i, row in Venuesdf.iterrows():
    avgdist=(((Participantsdf[Participantsdf['Venue']==row['Venue']])['Distance']).median())
    Venuesdf.loc[i,'AvgDistance']=avgdist 

# Mapping Average Travel Distance




We previously calculated the average travel distance for various off site activities. While this is really useful information it doesn't give any feeling for where these area in relationship to other events or activities or how close to the Gateshead Council boundary. Seeing this on a map makes it far more transparent. 

![Travel distance](Maps/traveldistance.png?raw=true)

The Folium Library is build on top of a javaScript library called Leaflet and by default circles in Leaflet take a radium in Metres which doesn't change as you zoom in and out and while there is talk on updating Folium to follow the same rules, the current default in for the size of the circles to change with zoom. The Size of the circles is therefore relative to the average travel distance on this plot rather than absolute. The learning team also feel that it would be more meaning ful if scale were related to number of participant rather than travel time.

In [ ]:
import folium
fr
om folium import plugins
from folium.plugins import MarkerCluster

# import data
Venuedf=pd.read_csv('Data/Venues.csv', sep=',')
Venuedf=Venuesdf[Venuedf['Venue']!="Baltic"]
Participantsdf=pd.read_csv("Data/All_Activities.csv", encoding = "ISO-8859-1", sep=',')

# create colour list
colour=['#FF00FF','#000080','#0000FF','#6A1B9A','#00FFFF','#00FF00','#FFFF00','#FF0000','#808080','#873600','#EF6C00','#3E2723','#827717','#689F38','#FFC107']

# create map
venuemap = folium.Map(location=[54.8721, -1.49828],zoom_start=10)

# L.circle([54.8721, -1.49828], {radius: 500}).addTo(venuemap);

# add venues
for i, row in Venuedf.iterrows():
        folium.CircleMarker(location=[row['Latitude'], row['Longitude']],popup=row['Venue'],radius=(row['AvgDistance']*16),color=colour[i],fill=True,fill_color=colour[i]).add_to(venuemap)

# save and display map
venuemap.save('Maps/foliumcluster.html')
venuemap

# Drawing conclusions on Exploratory Data Analysis with Maps

At the start of this exercise I mentioned that this project was based on real every 3 montha with colleagues from our Learning and Engagement team. In truth up until now the mapping element of that work i'd normally done with Google Maps, the the bulk of project usually being producng the statistical analysis on where participant come from and in particular how well represresented participant from Gateshead are in those numbers.

Although I have created cluser plots, heatmaps and ther mapping techniques before, this was the first time i'd used the Folium Library. I found Folium relatively easy to pick up, within a few lines of code you could rapidly build up map data that give you really useful insites into were individual participants were from, where clusters of participants were to be found and whether there were any pockets of the local communinity that we arn't currently engaging with, all of which will help shape the future core gallery learning programme and off site activity. 

There are elements of the Folium Library that could be improved on, for example with Google maps it was much easier to add a kml outline of the council boundry. With Folium you have to export the file as geojson and add that.  